In [32]:
# загрузка данных Мельбурн

import pandas as pd

#melb_df = pd.read_csv('data/melb_data_fe.csv')
melb_df.head()

,Suburb,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom,...,Longtitude,Regionname,Propertycount,MeanRoomsSquare,AreaRatio,MonthSale,AgeBuilding,WeekdaySale,StreetType,Weekend
0,Abbotsford,2,house,1480000.0,S,Biggin,2016-03-12,2.5,3067,2,...,144.9984,Northern Metropolitan,4019,25.20,-0.231707,3,46,5,St,1
1,Abbotsford,2,house,1035000.0,S,Biggin,2016-04-02,2.5,3067,2,...,144.9934,Northern Metropolitan,4019,15.80,-0.327660,4,116,5,St,1
2,Abbotsford,3,house,1465000.0,SP,Biggin,2017-04-03,2.5,3067,3,...,144.9944,Northern Metropolitan,4019,18.75,0.056338,4,117,0,St,0
3,Abbotsford,3,house,850000.0,PI,Biggin,2017-04-03,2.5,3067,3,...,144.9969,Northern Metropolitan,4019,15.75,0.145455,4,47,0,other,0
4,Abbotsford,4,house,1600000.0,VB,Nelson,2016-04-06,2.5,3067,3,...,144.9941,Northern Metropolitan,4019,17.75,0.083969,4,2,2,St,0


In [87]:
# сортировка данных

import pandas as pd

melb_df.sort_values(by='Price') # индексы сохранились из исходной таблицы
melb_df.sort_values(by='Date', ascending=False) # сортировка по убыванию
melb_df.sort_values(by=['Distance', 'Price']).loc[::10, ['Distance', 'Price']] # сортировка по несольким столбцам

mask1 = melb_df['AreaRatio'] < -0.8
mask2 = melb_df['Type'] == 'townhouse'
mask3 = melb_df['SellerG'] == 'McGrath'
melb_df[mask1 & mask2 & mask3].sort_values( # фильтрация
    by=['Date', 'AreaRatio'], # сортировка
    ascending=[True, False], # порядок для каждого столбца
    ignore_index=True # замена старых индексов на новые
).loc[:, ['Date', 'AreaRatio']] 

# фильтрация х2, сортировка х2, замена индекса индекс, доступ к ячейке
melb_df[(melb_df['Type'] == 'townhouse') & (melb_df['Rooms'] > 2)].sort_values(
    by=['Rooms','MeanRoomsSquare'], ascending=[True,False], ignore_index=True).loc[18,'Price']


1300000.0

In [95]:
# группировка данных

melb_df.groupby(by='Type').mean() # группы стоят в качестве индекса
melb_df.groupby(by='Type', as_index=False).mean() # отображение групп в качестве отдельного столбца
melb_df.groupby('Type')['Price'].mean() # объект типа Series
melb_df.groupby('Regionname')['Distance'].min().sort_values(ascending=False) # сортировка после группировки

melb_df.groupby('MonthSale')['Price'].agg(['count', 'mean', 'max'] # несколько агрегирующих функций
).sort_values(by='count', ascending=False) # объект типа Dataframe
melb_df.groupby('MonthSale')['Price'].agg('describe') # все основные стат характеристики

melb_df.groupby('Regionname')['SellerG'].agg('nunique') # подсчет уникальных значений
melb_df.groupby('Regionname')['SellerG'].agg(['nunique', set]) # выдача уникальных значений в виде множества

# фильтрация х2, группировка, сумма, сортировка х2
melb_df[(melb_df['Date'] >= '2017-05-01') & (melb_df['Date'] <= '2017-09-01')].groupby(by='SellerG')['Price'].sum().sort_values()

,nunique,set
Regionname,,
Eastern Metropolitan,26,"{RT, Love, Gary, Fletchers, McGrath, Barry, Ka..."
Eastern Victoria,11,"{HAR, Barry, Harcourts, C21, other, Ray, O'Bri..."
Northern Metropolitan,40,"{RT, Greg, Love, Raine, Rendina, McDonald, Ale..."
Northern Victoria,11,"{HAR, Barry, Buckingham, Raine, McDonald, YPA,..."
South-Eastern Metropolitan,25,"{Greg, Gary, O'Brien, Fletchers, Eview, McGrat..."
Southern Metropolitan,38,"{RT, Greg, Raine, Rendina, Gary, Nick, O'Brien..."
Western Metropolitan,34,"{RT, Greg, Douglas, McDonald, Rendina, Raine, ..."
Western Victoria,6,"{HAR, Raine, YPA, other, Ray, hockingstuart}"


In [156]:
# сводные таблицы

melb_df.groupby(['Rooms', 'Type'])['Price'].mean().unstack() # группировка по нескольким признакам (первый - строки, второй — столбцы)

melb_df.pivot_table( # создание сводной таблицы
    values='Landsize', 
    index='Regionname', # имя или список строк
    columns='Type', # имя или список столбцов
    aggfunc=['median', 'mean'], # имя или список агрегирующих функций (по умолчанию — mean)
    fill_value=0 # значение, которым необходимо заполнить пропуски
).round() # округление до целых

pivot = melb_df.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median', 'mean'],
    fill_value=0
)
pivot.columns # объекта типа MultiIndex, хранит все возможные варианты обращения к столбцам
pivot['mean']['unit'] # последовательное обращение к столбцам
filtered_pivot = pivot[pivot['mean']['house'] < pivot['median']['house']] # фильтрация внутри сводной
list(filtered_pivot.index) # индексы сводной

['Southern Metropolitan', 'Western Metropolitan']

In [224]:
# загрузка данных Фильмы

import pandas as pd

ratings1 = pd.read_csv('data/ratings1.csv')
ratings2 = pd.read_csv('data/ratings2.csv')
movies = pd.read_csv('data/movies.csv')
dates = pd.read_csv('data/dates.csv')

In [228]:
# объединение таблиц

ratings = pd.concat( # присоединение таблицы снизу
    [ratings1, ratings2],
    ignore_index=True # назначение нового сквозного индекса
)

ratings = ratings.drop_duplicates(ignore_index=True) # удаление дублирующихся строк и переназначение индекса

ratings_dates = pd.concat([ratings, dates], axis=1) # присоединение таблицы справа
